# Homework 05: Public key encryption

This homework will focus on topics covered in the final week of the course: public key cryptography.

## Helper functions 
To get you started, run the cell below to load functions that have been written in earlier homework assignments. Functions that are included:

* `gcd(<a>,<b>)`. This function will compute the greatest common divisor between a and b
* `mod_inverse(<e>, <n>)`. Computes the multiplicative inverse of e in modulus n
* `phi(<n>)`. This is Euler’s totient function as described in 11.3 RSA

You can use these functions in any of your code below after you’ve imported them.

In [18]:
from hw05toolkit import *

## 1. KidRSA 
Recommended Reading: https://www.macs4200.org/chapters/11/1/kidrsa.html

In this question you will implement the KidRSA algorithm.

Part A: Key Generation 
Write a function kidrsa_keygen that uses the values for a, b, a_prime, and b_prime to compute a pair of public and private keys using the Kid RSA key generation algorithm. The function will return the pair of keys as a list of lists. The first key in the list should be the public key  (𝑒,𝑛)
  and the second key in the list should be the private key  (𝑑,𝑛)
 .

**Sample Test Cases:**
```
>>> kidrsa_keygen(3,7,5,11)
[[103, 1169], [227, 1169]]

>>> kidrsa_keygen(2,324,100,8)
[[64702, 550017], [5500, 550017]]
```

In [2]:
def kidrsa_keygen(a, b, a_prime, b_prime):
    # BEGIN SOLUTION
    M = a * b - 1
    e = a_prime * M + a
    d = b_prime * M + b
    n = int((e * d - 1)/M)
    return [ [e, n], [d, n] ]
# END SOLUTION

In [13]:
kidrsa_keygen(2,324,100,8)

[[64702, 550017], [5500, 550017]]

In [14]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are returning values, not printing them
points: 0
""" # END TEST CONFIG
kidrsa_keygen(23, 34, 45, 56) != None

True

In [15]:
""" # BEGIN TEST CONFIG
failure_message: Review 11.1 in the textbook to make sure you are calculating M, e, d, and n correctly.
points: 1
""" # END TEST CONFIG
kidrsa_keygen(23, 34, 45, 56)[1][0]

43770

In [16]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are calculating the integer of n. If you divide by a value, python will always return a float by default.
points: 1
""" # END TEST CONFIG
kidrsa_keygen(4123, 3984, 1145, 5067)

[[18807809618, 95299175896087], [83230703061, 95299175896087]]

### Part B: Encryption / Decryption

Write a function `kidrsa` that takes in a numerical integer message, a list containing either a public or private key, and encrypts or decrypts the message. Since the process for encryption and decryption are identical, you do not need a boolean `encipher` parameter as in earlier homework assignments and labs.

`kidrsa` should output a numerical integer that represents ciphertext when the function is provided numerical plaintext, and vice-versa.

**Reminder:** Your message, `message`, must be less than the modulus `n` from the key **and** be relatively prime to `n` (i.e `gcd(m,n) == 1`). Your function must check for this condition and return the boolean value `False` if it does not meet these criteria.

**Sample Test Cases**:
```
>>> kidrsa( 348, [18221, 347273])
89994

>>> kidrsa( 89994, [9377, 347273])
348
```

If `message` $\gt n$
```
>>> kidrsa( 500000, [18221, 347273])
False
```

If `gcd(message, n)` $\neq 1$
```
>>> kidrsa( 61, [18221, 347273])
False
```

_Type your answer here, replacing this text._

In [3]:
def kidrsa(message, key, encrypt=True):
    
    # BEGIN SOLUTION
    e_or_d = key[0]
    n = key[1]
    if message >= n:
        return False
    if gcd(message, n) != 1:
        return False
    plaintext = (message * e_or_d) % n
    return (plaintext)

# END SOLUTION

In [33]:
# You test your function here

In [34]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are returning and not printing.
points: 1
""" # END TEST CONFIG
kidrsa(93,[17,101]) != None

True

In [35]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are multiplying and modding by the right elements in the key.
points: 1
""" # END TEST CONFIG
kidrsa(93,[17,101])

66

In [38]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are returning False if the message and the mod are not relatively prime.
""" # END TEST CONFIG
kidrsa(90,[11,100])

False

### Part C: Cracking Keys

KidRSA is not secure because it's fairly easy and fast to determine a private key if you have someone's private key. Write a function `crack_kidrsa` that accepts a public key (as a list) as the input, and returns the private key.

`crack_kidrsa` must return an a list that represents the private key formatted as `[d, n]`. `crack_kidrsa` must check to ensure that the provided value of `e` has an inverse in mod `n`. If they aren't the function should return the boolean value `False`

**HINT:** You should not brute force this! You should use the `mod_inverse` function to help here. `mod_inverse` returns the boolean value `False` if there is no multiplicative inverse found. 

**Sample Test Cases**:
```
>>> crack_kidrsa( [18221, 347273] )
[9377, 347273]

>>> crack_kidrsa( [4, 12] )
False
```

In [39]:
def crack_kidrsa(public_key):
    e = public_key[0] # SOLUTION
    n = public_key[1] # SOLUTION
    d = mod_inverse(e,n) # SOLUTION
    # BEGIN SOLUTION
    
    if d == False:
        return False
    if n <= e:
        return False
    
    
    
    return [d,n]
# END SOLUTION

Use the empty cell below to put your function through a few tests of your own, then run the autograder cell.

In [40]:
# You test your function here

In [41]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are returning and not printing.
points: 1
""" # END TEST CONFIG
crack_kidrsa([17,101]) != None

True

In [42]:
""" # BEGIN TEST CONFIG
failure_message: Check that you are taking in the right part of the key as the mod and the encipher value.
points: 1
""" # END TEST CONFIG
crack_kidrsa([17,101])

[6, 101]

In [43]:
""" # BEGIN TEST CONFIG
failure_message: Use a conditional so your program returns False if the key and mod are not relatively prime or if the mod is smaller than the key.
points: 1
""" # END TEST CONFIG
crack_kidrsa([820003, 5001]) == crack_kidrsa([100,200]) and crack_kidrsa([100,200]) == False

True

## 2. The RSA Cipher

*Recommended Reading:* https://www.macs4200.org/chapters/11/3/rsa.html

### Part A: Key Generation

Write a function `rsa_keygen` that uses the prime integers `p` and `q`, and the encryption exponent `e` to compute a pair of public and private keys using the RSA key generation algorithm. `e` should take on the default value of `65537`. 

The function must return the pair of keys as a list of lists. The first key in the list should be the public key $\left(e,n\right)$ and the second key in the list should be the private key $\left(d,n\right)$.

`rsa_keygen` must check that `e` is a valid encryption exponent. If it's not, `kid_rsa` should return the boolean value `False`

**Reminder:**  $e$ and $\varphi(n)$ need to be relatively prime for $e$ to be valid.

**Sample Test Cases**:
```
>>> rsa_keygen(3, 11, 3)
[(3, 33), (7, 33)]

>>> rsa_keygen(1787, 3881)
[(65537, 6935347), (5655233, 6935347)]

>>> rsa_keygen(1787, 3881, e=36)
False
```

In [16]:
def rsa_keygen(p, q, e=65537):
    
    
    n = p*q # SOLUTION
    my_phi = (p-1)*(q-1) # SOLUTION
    d = mod_inverse(e,my_phi) % my_phi # SOLUTION
    
    if gcd(e, my_phi) != 1: # SOLUTION NO PROMPT
        return False  # SOLUTION NO PROMPT
     
    
    return [ [e,n], [d,n] ]

In [19]:
""" # BEGIN TEST CONFIG
failure_message: Remember to return, not print.
points: 1
""" # END TEST CONFIG
rsa_keygen(13, 11, 17) != None

True

In [32]:
""" # BEGIN TEST CONFIG
failure_message: You may wish to check your mod_inverse program or check that you calculated phi correctly.
points: 1
""" # END TEST CONFIG
rsa_keygen(23,73, 233)

[[233, 1679], [809, 1679]]

In [33]:
""" # BEGIN TEST CONFIG
failure_message: Use a conditional so your program returns False if e and phi are not relatively prime.
points: 1
""" # END TEST CONFIG
rsa_keygen(33,73, 234)

False

Use the empty cell below to put your function through a few tests of your own, then run the autograder cell.

In [ ]:
# You test your function here


### Part B: Encryption / Decryption

Write a function `rsa` that takes in a numerical integer message, a list containing either a public or private key, and encrypts or decrypts the message. Since the process for encryption and decryption are identical, you do not need a boolean `encipher` parameter as in earlier homework assignments and labs.

`rsa` should output a numerical integer that represents ciphertext when the function is provided numerical plaintext, and vice-versa.

**Reminder:** Your message, `message`, must be less than the modulus `n` from the key. Your function must check for this condition and return the boolean value `False` if it does not meet the criteria.

**Hint:** You should use the `pow` function instead of the `**` operator so you're using the fast modular exponentiation algorithm. This will drastically reduce the time to encrypt or decrypt a message. The numbers in this example are far too large for Python to efficiently carry out the exponentiation with the `**` operator!

**Sample Test Cases**:
```
>>> rsa( 5, [3,33] )
26

>>> rsa( 26, (7,33) )
5

>>> rsa( 234712, [65537, 6935347])
2678654

>>> rsa( 2678654, [5655233, 6935347])
234712

m > n case
>>> rsa( 26786543243423, [5655233, 6935347])
False
```

In [20]:
def rsa(message, key):
    e_or_d = key[0] # SOLUTION
    n = key[1] # SOLUTION
    
    # BEGIN SOLUTION
    
    if message > n:
        return False
    
    
    
    return pow(message, e_or_d, n)

# END SOLUTION

Use the empty cell below to put your function through a few tests of your own, then run the autograder cell.

In [10]:
# You test your function here

In [11]:
""" # BEGIN TEST CONFIG
failure_message: Remember to return, not print.
points: 1
""" # END TEST CONFIG
rsa(5, [67,91])

47

In [12]:
""" # BEGIN TEST CONFIG
failure_message: Remember to use pow and not **.
points: 1
""" # END TEST CONFIG
rsa(9329802, [4103916,9966745])

729161

In [14]:
""" # BEGIN TEST CONFIG
failure_message: Remember to return false if the message is larger than the mod.
points: 1
""" # END TEST CONFIG
rsa(1729493, [1919281, 17])

False